# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.15it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.80it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.43it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.26it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  6.39it/s]

 78%|███████▊  | 18/23 [00:03<00:00,  6.36it/s]

 87%|████████▋ | 20/23 [00:03<00:00,  6.49it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  6.18it/s]

100%|██████████| 23/23 [00:04<00:00,  5.17it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Avery and I'm a junior at WCC. I'm an undecided major right now, but I'm hoping to declare a major in media studies or possibly communications. I'm really passionate about storytelling and I love exploring different ways to express myself creatively.
When I'm not in class, you can usually find me at the student union playing pool or working on my own writing projects. I'm really interested in social justice and I try to stay up-to-date on current events and politics. I'm also a bit of a film buff and I love watching independent movies and documentaries.
I'm excited to be a part of this community and I'm
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States. The president serves a four-year term and is elected by the citizens of the United States through the Electoral College system.
The president is responsible for executing the laws of the United States, as well as for set

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Sylvia

 Sylvia and

 Sylvia and I

 Sylvia and I am

...

 uncertain and will likely be shaped by various factors, including technological advancements, societal values, and economic interests. Here are some potential trends and predictions for the future of AI:
1. Increased focus on explainability and transparency: As AI becomes more ubiquitous, there will be a growing need for AI systems to be explainable and transparent in their decision-making processes.
2. Advancements in natural language processing: NLP will continue to improve, enabling AI systems to better understand and generate human language, leading to more sophisticated chatbots and virtual assistants.
3. Rise of edge AI: As IoT devices and sensor networks become more prevalent

 uncertain and will likely be shaped by various factors, including technological advancements, societal values, and economic interests. Here are some potential trends and predictions for the future of AI:
1. Increased focus on explainability and transparency: As AI becomes more ubiquitous, there will be a growing need for AI systems to be explainable and transparent in their decision-making processes.
2. Advancements in natural language processing: NLP will continue to improve, enabling AI systems to better understand and generate human language, leading to more sophisticated chatbots and virtual assistants.
3. Rise of edge AI: As IoT devices and sensor networks become more prevalent,

 uncertain and will likely be shaped by various factors, including technological advancements, societal values, and economic interests. Here are some potential trends and predictions for the future of AI:
1. Increased focus on explainability and transparency: As AI becomes more ubiquitous, there will be a growing need for AI systems to be explainable and transparent in their decision-making processes.
2. Advancements in natural language processing: NLP will continue to improve, enabling AI systems to better understand and generate human language, leading to more sophisticated chatbots and virtual assistants.
3. Rise of edge AI: As IoT devices and sensor networks become more prevalent, edge

 uncertain and will likely be shaped by various factors, including technological advancements, societal values, and economic interests. Here are some potential trends and predictions for the future of AI:
1. Increased focus on explainability and transparency: As AI becomes more ubiquitous, there will be a growing need for AI systems to be explainable and transparent in their decision-making processes.
2. Advancements in natural language processing: NLP will continue to improve, enabling AI systems to better understand and generate human language, leading to more sophisticated chatbots and virtual assistants.
3. Rise of edge AI: As IoT devices and sensor networks become more prevalent, edge AI

 uncertain and will likely be shaped by various factors, including technological advancements, societal values, and economic interests. Here are some potential trends and predictions for the future of AI:
1. Increased focus on explainability and transparency: As AI becomes more ubiquitous, there will be a growing need for AI systems to be explainable and transparent in their decision-making processes.
2. Advancements in natural language processing: NLP will continue to improve, enabling AI systems to better understand and generate human language, leading to more sophisticated chatbots and virtual assistants.
3. Rise of edge AI: As IoT devices and sensor networks become more prevalent, edge AI will

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Jessica and I'm a senior editor at Poptropica, working on content creation, including game scripts, dialogue, and narrative design. I'm excited to share my experiences and insights with you.

I'm interested in learning more about the community's thoughts on character development, game world design, and storytelling in Poptropica. How do you think these elements contribute to the game's overall magic and appeal?

Also, I'd love to hear about your favorite Poptropica islands and what makes them special to you. Do you have any favorite characters or storylines that you'd like to see revisited or expanded upon in future

Prompt: The capital of France is
Generated text:  home to the world-famous Eiffel Tower, but it has so much more to offer. From art and history to fashion and cuisine, Paris has something for everyone. Here are some must-see attractions and experiences to add to your itinerary:
The Eiffel Tower: The iconic iron lattice tower is a

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Fil

 Filipe

 Filipe


 Filipe
I

...

 here. What does it mean for us?
Artificial Intelligence (AI) is no longer just a futuristic concept – it’s a reality that’s transforming industries, societies, and our individual lives. As AI becomes increasingly integrated into our daily experiences, we must consider its implications, both positive and negative. Let’s explore the future of AI and its potential impact on us.
What is the current state of AI?
AI has made tremendous progress in recent years, with advancements in machine learning, natural language processing, and computer vision. AI systems can now perform complex tasks, such as:
  1. Image and speech recognition

 here. What does it mean for us?
Artificial Intelligence (AI) is no longer just a futuristic concept – it’s a reality that’s transforming industries, societies, and our individual lives. As AI becomes increasingly integrated into our daily experiences, we must consider its implications, both positive and negative. Let’s explore the future of AI and its potential impact on us.
What is the current state of AI?
AI has made tremendous progress in recent years, with advancements in machine learning, natural language processing, and computer vision. AI systems can now perform complex tasks, such as:
  1. Image and speech recognition


 here. What does it mean for us?
Artificial Intelligence (AI) is no longer just a futuristic concept – it’s a reality that’s transforming industries, societies, and our individual lives. As AI becomes increasingly integrated into our daily experiences, we must consider its implications, both positive and negative. Let’s explore the future of AI and its potential impact on us.
What is the current state of AI?
AI has made tremendous progress in recent years, with advancements in machine learning, natural language processing, and computer vision. AI systems can now perform complex tasks, such as:
  1. Image and speech recognition
 

 here. What does it mean for us?
Artificial Intelligence (AI) is no longer just a futuristic concept – it’s a reality that’s transforming industries, societies, and our individual lives. As AI becomes increasingly integrated into our daily experiences, we must consider its implications, both positive and negative. Let’s explore the future of AI and its potential impact on us.
What is the current state of AI?
AI has made tremendous progress in recent years, with advancements in machine learning, natural language processing, and computer vision. AI systems can now perform complex tasks, such as:
  1. Image and speech recognition
  

 here. What does it mean for us?
Artificial Intelligence (AI) is no longer just a futuristic concept – it’s a reality that’s transforming industries, societies, and our individual lives. As AI becomes increasingly integrated into our daily experiences, we must consider its implications, both positive and negative. Let’s explore the future of AI and its potential impact on us.
What is the current state of AI?
AI has made tremendous progress in recent years, with advancements in machine learning, natural language processing, and computer vision. AI systems can now perform complex tasks, such as:
  1. Image and speech recognition
  2

In [6]:
llm.shutdown()